# Hi Splitter: Under the Hood
If you want to modify the Hi Splitter or find it too slow, this tutorial offers a high-level overview of how the algorithm functions. By the end, you'll be equipped to read and modify the source code, which spans only 500 lines.

Before diving into this tutorial, it's essential to be familiar with 00_hi_split_quick.ipynb, 01_hi_split_coarsening.ipynb, and the paper on the LoHi benchmark.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import mip

import lohi_splitter as lohi

In [2]:
# We will use a small subset of drd2
drd2_hi = pd.read_csv('data/drd2_hi.csv', index_col=0)

# Take random subset of 400 molecules
idx = np.arange(len(drd2_hi))
np.random.shuffle(idx)
idx = idx[:400]
drd2_hi = drd2_hi.iloc[idx]
drd2_hi

,smiles,value
6040,O[C@]1(c2cccc(Cl)c2Cl)C[C@@H]2CC[C@H](C1)N2Cc1...,True
2315,COc1ccc(Cl)cc1S(=O)(=O)N[C@H]1CC[C@H](N2CCC(c3...,True
1539,CN1CCC(C(=O)N[C@H]2CC[C@H](CCN3CCC(c4cccc5c4OC...,True
2163,COc1cc2c(cc1OC)-c1c(OC)c(O)c(I)c3c1[C@H](C2)N(...,True
1625,CN1CCN(C2=Nc3cc(Cl)ccc3N(NC(=O)c3cccnc3)c3cccc...,False
...,...,...
382,CC(C)Oc1ccccc1N1CCN(Cc2ccc(CN3CCCCC3=O)n2C)CC1,True
890,CCCN(CCN1CCN(c2ccc3[nH]ccc3c2)CC1)C1CCc2c(O)cc...,True
4588,O=C(C[C@@H]1COCCO1)N[C@H]1CC[C@H](CCN2CCN(c3nc...,False
5686,O=S(=O)(c1ccc(-n2cccn2)cc1)C1(F)CCN(CCc2ccc(F)...,False


# Hi-Splitter, No Coarsening 

In [3]:
smiles = drd2_hi['smiles'].to_list()

# Create neighborhood graph
neighborhood_graph = lohi.get_neighborhood_graph(smiles, threshold=0.4)

In [4]:
# To efficiently handle both the giant connectivity component and the smaller components, we need to separate them:
main_component, small_components = lohi.get_giant_component(neighborhood_graph)
main_component

In [5]:
# The `main_component` is a subgraph and retains the same node indices as the original graph. 
# These indices can be challenging to work with, so let's renumber the nodes.
old_nodes_to_new = dict(
    zip(main_component.nodes(), range(main_component.number_of_nodes()))
)
new_nodes_to_old = {v: k for k, v in old_nodes_to_new.items()}
main_component = nx.relabel_nodes(main_component, old_nodes_to_new)

# We'll use `new_nodes_to_old` later to revert to the node indices of the original graph.

In [6]:
# The Hi-splitter employs the MIP library to tackle linear programming.
# The following function formulates the linear problem, returning a mip.model.Model object.
model = lohi.get_linear_problem_train_test(
    main_component, train_min_frac=0.7, test_min_frac=0.1, verbose=True
)
model

Total molecules in the giant component: 211.0
Min train size 147
Min test size 21


In [7]:
# Now, it's time to solve the linear problem. 
# We could use a function like: `solve_linear_problem(model, max_mip_gap, verbose)`
# However, for clarity, let's unpack the function's operations.

# These lines set optimization parameters. Refer to the MIP documentation for detailed explanations of each parameter.
model.max_mip_gap = 0.1
model.threads = -1
model.emphasis = 2
model.verbose = True
model.optimize()


Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 2007 (-422) rows, 422 (0) columns and 4432 (-844) elements
Clp1000I sum of infeasibilities 3.52275e-07 - average 1.75523e-10, 0 fixed columns
Coin0506I Presolve 2007 (0) rows, 422 (0) columns and 4432 (0) elements
Clp0029I End of values pass after 422 iterations
Clp0014I Perturbing problem by 0.001% of 1.0686703 - largest nonzero change 2.9969234e-05 ( 0.0014984617%) - largest zero change 0
Clp0000I Optimal - objective value 211
Clp0000I Optimal - objective value 211
Clp0000I Optimal - objective value 211
Coin0511I After Postsolve, objective 211, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 211 - 0 iterations time 0.042, Presolve 0.00, Idiot 0.04

Starting MIP optimization
Cgl0004I processed model has 2007 rows, 422 columns (422 integer (422 of which binary)) and 4432 elements
Coin3009W 

<OptimizationStatus.OPTIMAL: 0>

In [8]:
# Now we have results
result = np.array([a.x for a in model.vars])
result

array([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0.

In [10]:
"""
The `result` provides a solution to the linear problem. Considering there are twice as many variables as there are 
molecules in the giant component, these variables serve as indicators for the placement of the i'th molecule in the 
k'th partition. Given that this is a train-test split with k=2:

- `results[0]` is 1 if the 0'th molecule of the giant component is in the training set.
- `results[1]` is 1 if the 0'th molecule of the giant component is in the testing set.
- `results[2]` is 1 if the 1'st molecule of the giant component is in the training set.
... and so on.

It's important to note that a molecule cannot belong to both the training and testing sets simultaneously. 
However, a molecule might belong to neither, implying it has been discarded.
"""


"\nThe `result` provides a solution to the linear problem. Considering there are twice as many variables as there are \nmolecules in the giant component, these variables serve as indicators for the placement of the i'th molecule in the \nk'th partition. Given that this is a train-test split with k=2:\n\n- `results[0]` is 1 if the 0'th molecule of the giant component is in the training set.\n- `results[1]` is 1 if the 0'th molecule of the giant component is in the testing set.\n- `results[2]` is 1 if the 1'st molecule of the giant component is in the training set.\n... and so on.\n\nIt's important to note that a molecule cannot belong to both the training and testing sets simultaneously. \nHowever, a molecule might belong to neither, implying it has been discarded.\n"

In [12]:
# To make the results more intuitive, let's transform the result into a simpler format.
result = np.array([a.x for a in model.vars])
split = [-1] * len(main_component)

# Iterate through the molecules in the giant component
for i in range(len(main_component)):
    # Check in which partition (train or test) the molecule resides
    for j in range(2):
        if result[i * 2 + j]:
            split[i] = j
            break
split

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

In [13]:
# Alternatively, we can utilize the `process_results` function to directly obtain the split.
split = lohi.process_results(model=model, S=main_component, k=2)
split

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

In [14]:
# With the partitions determined for the nodes of the giant component,
# it's time to map these partitions back to the original graph's indices.
partitions = lohi.map_split_to_original_idx(
    split, k=2, new_nodes_to_old=new_nodes_to_old
)

# Next, we'll incorporate the small components into the partitioning.
partitions = lohi.assign_small_components_train_test(small_components, partitions, train_min_frac=0.7, test_min_frac=0.1)

In [15]:
train = drd2_hi.iloc[partitions[0]]
test = drd2_hi.iloc[partitions[1]]
train

,smiles,value
1539,CN1CCC(C(=O)N[C@H]2CC[C@H](CCN3CCC(c4cccc5c4OC...,True
4921,O=C(N[C@H]1CC[C@H](CCN2CCC(c3cccc4c3CCO4)CC2)C...,True
4585,O=C(C[C@@H]1COCCO1)N[C@H]1CC[C@H](CCN2CCC(c3cc...,True
4953,O=C(N[C@H]1CC[C@H](CCN2CCC(c3cccc4c3OCO4)CC2)C...,True
4987,O=C(N[C@H]1CC[C@H](CCN2CCC(c3coc4ccccc34)CC2)C...,True
...,...,...
1101,CCCNC1CCc2cccc(OS(=O)(=O)C(F)(F)F)c2C1,False
863,CCCN(CCN1CCN(Cc2cc3ccccc3[nH]2)CC1)c1ccc(Br)cc1,True
963,CCCN(CC[C@]1(O)C[C@@H](NC(=O)c2ccc3ccccc3c2)C1...,True
4225,N#Cc1ccc2c(c1)CNCC2,False
